In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%reload_ext autoreload
%autoreload 2

experiment_path = 'experiments/UWB/'

#plt.rcParams['figure.figsize'] = 7, 3
#matplotlib.rcParams['pdf.fonttype'] = 42
#matplotlib.rcParams['ps.fonttype'] = 42

# set the random seed so can reproduce when something didn't work. 
# (only when cells are run in order)
np.random.seed(1)

In [ ]:
experiment_dir = 'experiments/UWB/uwb_ball_29-12-2019/'
fname = 'eights'

In [ ]:
# Extract the multiindex from somewhat messy Optitrack csv file header.
header_df = pd.read_csv(f"{experiment_dir}/{fname}.csv", low_memory=False, skiprows=range(2), header=None)[:5]
names = header_df.iloc[0, :].values.astype(str)
ids = header_df.iloc[1, :].values.astype(str)
types = header_df.iloc[3, :].values.astype(str)
labels = header_df.iloc[4, :].values.astype(str)
valid_indices = [j for j, name_ in enumerate(names) if name_ in ['Rigid Body', 'Rigid Body Marker', 'nan']]
tuples = list(tuple(zip(names, ids, types, labels)))
tuples = [t for i, t in enumerate(tuples) if i in valid_indices]
index = pd.MultiIndex.from_tuples(tuples)

In [ ]:
# Read csv file and change index accordingly

data_df = pd.read_csv(f"{experiment_dir}/{fname}.csv", skiprows=range(7), usecols=valid_indices)

# change columns to multiindex
data_df.columns = index

# change all numeric columns from default double to single precision.
data_df = data_df.apply(pd.to_numeric, downcast='float', errors='ignore', axis=1)

# change index to frames and times
frames = data_df["nan", "nan", "nan", "Frame"].values
times = data_df["nan", "nan", "nan", "Time (Seconds)"].values
index = pd.MultiIndex.from_tuples(list(zip(frames, times)), names=['frame', 'time'])
data_df.index = index
data_df.drop(columns='nan', inplace=True)

# drop useless columns
drop_columns = [c for c in data_df.columns if ("Marker Quality" in c) or ("Error Per Marker" in c)]
data_df.drop(columns=drop_columns, inplace=True)

data_df

In [ ]:
test_x = data_df.loc[:, ("Rigid Body Marker", "default 01:Marker1", "Position", "X")].values
test_y = data_df.loc[:, ("Rigid Body Marker", "default 01:Marker1", "Position", "Y")].values
test_z = data_df.loc[:, ("Rigid Body Marker", "default 01:Marker1", "Position", "Z")].values

test_x = data_df.loc[:, ("Rigid Body", "default 01", "Position", "X")].values
test_y = data_df.loc[:, ("Rigid Body", "default 01", "Position", "Y")].values
test_z = data_df.loc[:, ("Rigid Body", "default 01", "Position", "Z")].values

print(test_x, test_y)
print('all nan:', all(np.isnan(test_x)))
print('all nan:', all(np.isnan(test_y)))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(test_x, test_y)
fig, ax = plt.subplots()
ax.scatter(test_x, test_z)
fig, ax = plt.subplots()
ax.scatter(test_y, test_z)